In [34]:
import pandas as pd
import numpy as np
import math
from geopy.geocoders import Nominatim
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

### Map monde, sympa pour l'oral, remplacer 'equirectangular' par 'orthographic'

In [35]:
#Plotly with jupyter offline : 
init_notebook_mode(connected=True)

In [36]:
df = pd.read_csv('map.csv', index_col=0, names = ['nb_attack', 'target', 'attacker'], header = 0)

In [37]:
df.head()

,nb_attack,target,attacker
0,92157,united states,united states
1,26877,united arab emirates,china
2,13059,united states,china
3,6424,united states,ukraine
4,5202,united arab emirates,netherlands


In [38]:
df_country = pd.DataFrame(list(set(list(df['attacker'].unique()) + list(df['target'].unique()))), columns = ['country'] )

In [39]:
geolocator = Nominatim()
df_country['lattitude_longitude'] = df_country['country'].apply(lambda x : geolocator.geocode(x,  timeout=10)[1])

In [40]:
# Geolocator is wrong for united arabe emirates 
df_country['lattitude_longitude'] = df_country.apply(lambda x : (25.61552, 55.84951) if x['country'] == 'united arab emirates' else x['lattitude_longitude'], axis = 'columns')
df_country['lattitude_longitude'] = df_country.apply(lambda x : (39.904211, 116.407395) if x['country'] == 'china' else x['lattitude_longitude'], axis = 'columns')
df_country['lattitude_longitude'] = df_country.apply(lambda x : (37.566535, 126.9779692) if x['country'] == 'south korea' else x['lattitude_longitude'], axis = 'columns') 
df_country['lattitude_longitude'] = df_country.apply(lambda x : (51.3030, 0.0732) if x['country'] == 'united kingdom' else x['lattitude_longitude'], axis = 'columns') 
df_country['lattitude_longitude'] = df_country.apply(lambda x : (40.18658704808404, 44.78151767883003) if x['country'] == 'armenia' else x['lattitude_longitude'], axis = 'columns') 

In [41]:
df_country['latitude_attacker'] = df_country['lattitude_longitude'].apply(lambda x : x[0])
df_country['longitude_attacker'] = df_country['lattitude_longitude'].apply(lambda x : x[1])
df_country['attacker'] = df_country['country']
df_country = df_country.drop(['lattitude_longitude', 'country'], axis = 'columns')

In [42]:
df = pd.merge(df, df_country, how = 'left', on = ['attacker'])

In [43]:
df_country['latitude_target'] = df_country['latitude_attacker']
df_country['longitude_target'] = df_country['longitude_attacker']
df_country['target'] = df_country['attacker']
df_country = df_country.drop(['longitude_attacker', 'attacker', 'latitude_attacker'], axis = 'columns')

In [44]:
df = pd.merge(df, df_country, how = 'left', on = ['target'])

In [45]:
df.head()

,nb_attack,target,attacker,latitude_attacker,longitude_attacker,latitude_target,longitude_target
0,92157,united states,united states,39.783730,-100.445882,39.78373,-100.445882
1,26877,united arab emirates,china,39.904211,116.407395,25.61552,55.849510
2,13059,united states,china,39.904211,116.407395,39.78373,-100.445882
3,6424,united states,ukraine,49.487197,31.271832,39.78373,-100.445882
4,5202,united arab emirates,netherlands,52.237989,5.534607,25.61552,55.849510


In [46]:
df['lat'] = df.apply(lambda x :[x['latitude_attacker'], x['latitude_target']], axis ='columns')
df['lon'] =  df.apply(lambda x :[x['longitude_attacker'], x['longitude_target']], axis ='columns')

In [47]:
df['line'] = ''
#df['line'] = df['line'].apply(lambda x : {'color' : 'red', 'width' : 1}) 
df['line'] = df.apply(lambda x : {'color' : 'red', 'width' : (x['nb_attack'] / sum(df['nb_attack']))*80}, axis='columns')

In [48]:
df['opacity'] = 0
df['opacity'] = df['opacity'].apply(lambda x : 0.48577680525164113)

In [49]:
df['type'] = ''
df['type'] = df['type'].apply(lambda x : 'scattergeo')

In [50]:
df['mode'] = ''
df['mode'] = df['mode'].apply(lambda x : 'lines')

In [51]:
df['text'] = df.apply(lambda x : '{} - {} : {} times'.format(x['attacker'],x['target'], x['nb_attack']), axis='columns')

In [52]:
df

,nb_attack,target,attacker,latitude_attacker,longitude_attacker,latitude_target,longitude_target,lat,lon,line,opacity,type,mode,text
0,92157,united states,united states,39.783730,-100.445882,39.783730,-100.445882,"[39.7837304, 39.7837304]","[-100.4458825, -100.4458825]","{'width': 34.083306365863805, 'color': 'red'}",0.485777,scattergeo,lines,united states - united states : 92157 times
1,26877,united arab emirates,china,39.904211,116.407395,25.615520,55.849510,"[39.904211, 25.61552]","[116.407395, 55.84951]","{'width': 9.940178447598354, 'color': 'red'}",0.485777,scattergeo,lines,china - united arab emirates : 26877 times
2,13059,united states,china,39.904211,116.407395,39.783730,-100.445882,"[39.904211, 39.7837304]","[116.407395, -100.4458825]","{'width': 4.829735102399334, 'color': 'red'}",0.485777,scattergeo,lines,china - united states : 13059 times
3,6424,united states,ukraine,49.487197,31.271832,39.783730,-100.445882,"[49.4871968, 39.7837304]","[31.2718321, -100.4458825]","{'width': 2.375849475290093, 'color': 'red'}",0.485777,scattergeo,lines,ukraine - united states : 6424 times
4,5202,united arab emirates,netherlands,52.237989,5.534607,25.615520,55.849510,"[52.2379891, 25.61552]","[5.53460738161551, 55.84951]","{'width': 1.9239055059867782, 'color': 'red'}",0.485777,scattergeo,lines,netherlands - united arab emirates : 5202 times
5,4761,spain,china,39.904211,116.407395,40.002803,-4.003104,"[39.904211, 40.0028028]","[116.407395, -4.003104]","{'width': 1.7608062502889372, 'color': 'red'}",0.485777,scattergeo,lines,china - spain : 4761 times
6,2754,united arab emirates,united states,39.783730,-100.445882,25.615520,55.849510,"[39.7837304, 25.61552]","[-100.4458825, 55.84951]","{'width': 1.0185382090518238, 'color': 'red'}",0.485777,scattergeo,lines,united states - united arab emirates : 2754 times
7,2523,united states,india,22.351115,78.667743,39.783730,-100.445882,"[22.3511148, 39.7837304]","[78.6677428, -100.4458825]","{'width': 0.9331052655910499, 'color': 'red'}",0.485777,scattergeo,lines,india - united states : 2523 times
8,2502,united states,south korea,37.566535,126.977969,39.783730,-100.445882,"[37.566535, 39.7837304]","[126.9779692, -100.4458825]","{'width': 0.9253386343673432, 'color': 'red'}",0.485777,scattergeo,lines,south korea - united states : 2502 times
9,2458,singapore,china,39.904211,116.407395,1.290475,103.852036,"[39.904211, 1.2904753]","[116.407395, 103.8520359]","{'width': 0.9090656927557672, 'color': 'red'}",0.485777,scattergeo,lines,china - singapore : 2458 times


In [221]:
df['showlegend'] = False
data_norse = df[['lat', 'line', 'lon', 'mode', 'opacity','type', 'showlegend']].to_dict(orient='records')
df_intra_pays = df[df['attacker'] == df['target']]

labels = [ dict(
        type = 'scattergeo',
        hoverinfo = 'text',
        lon = df['longitude_attacker'],
        lat = df['latitude_attacker'],
        text = df['attacker'],
        mode = 'markers',
        marker = dict( 
            size=2, 
            color='rgb(255, 0, 0)',
            line = dict(
                width=1,
                color='rgba(68, 68, 68, 0)'
            )   
        ),
        legendgroup = 'group1', # this can be any string, not just "group"
        name = 'Flux inter pays',
        showlegend = True
    
)]

cases = []
cases.append(go.Scattergeo(
    hoverinfo = 'text',
    lon = df_intra_pays['longitude_attacker'], 
    lat = df_intra_pays['latitude_attacker'],
    text = df_intra_pays['attacker'],
    marker = dict(
        size = df_intra_pays['nb_attack'].apply(lambda x :  x / sum(df['nb_attack']) * 80),
        color = 'rgb(33,113,181)',
        line = dict(width = 0)
    ),
    legendgroup = 'group2', # this can be any string, not just "group"
    name = 'Flux intra pays',
    mode = 'markers',
    showlegend = True
    
))

 
layout = dict(
    title = 'Flux des cyberattaques mondiale en mars 2018 <br> source : <a href="http://www.norse-corp.com/about-us/who-we-are/">\
Norse </a>',
        
        geo = dict(
            scope='world',
            projection= dict( type='orthographic'),
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
            showcountries = True
        )
    )
    
fig = dict(data = data_norse + cases + labels, layout=layout )
iplot(fig, filename='d3-world-map')

In [191]:
cases

[{'hoverinfo': 'text', 'lat': 0      39.783730
  22     46.603354
  23     17.726913
  29     22.350627
  35     64.686314
  50     42.638426
  53     34.982302
  55     51.083420
  102    51.303000
  112    40.033265
  124    19.432601
  140    12.750349
  Name: latitude_attacker, dtype: float64, 'legendgroup': 'group2', 'lon': 0     -100.445882
  22       1.888334
  23      42.236345
  29     114.184916
  35      97.745306
  50      12.674297
  53      33.145128
  55      10.423447
  102      0.073200
  112     -7.889626
  124    -99.133342
  140    122.731210
  Name: longitude_attacker, dtype: float64, 'marker': {'color': 'rgb(33,113,181)',
   'line': {'width': 0},
   'size': 0      34.083306
   22      0.404235
   23      0.371319
   29      0.308446
   35      0.258148
   50      0.167537
   53      0.158661
   55      0.155702
   102     0.055476
   112     0.052887
   124     0.049928
   140     0.026628
   Name: nb_attack, dtype: float64}, 'mode': 'markers', 'name': 'Flux intra

In [54]:
# 1ère cartes : Représentation des cyber attaques 
# 2 ème carte : cibles des cyber-attaques 
# 3 ème carte : origine des attaques 

In [55]:
df_code_country = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')[['CODE', 'COUNTRY']]
df_code_country['country'] = df_code_country['COUNTRY'].apply(lambda x : x.lower())
df_code_country = df_code_country.drop(['COUNTRY'], axis = 'columns')

In [56]:
sum_target = df.groupby(['target'])['nb_attack'].sum()
df_target = df.set_index(['target'])[['latitude_target', 'longitude_target']]
df_target['sum_target'] = sum_target
df_target = df_target.reset_index()
df_target = df_target.drop_duplicates('target')
df_target['country'] = df_target['target']
df_target = df_target.drop(['target'], axis = 'columns')

In [57]:
df_target = pd.merge(df_target, df_code_country, how = 'left', on = ['country'])

In [58]:
df_target.head()

,latitude_target,longitude_target,sum_target,country,CODE
0,39.783730,-100.445882,137702,united states,USA
1,25.615520,55.849510,45635,united arab emirates,ARE
2,40.002803,-4.003104,7740,spain,ESP
3,1.290475,103.852036,2702,singapore,SGP
4,64.573154,11.528036,1828,norway,NOR


In [59]:
sum_attacker = df.groupby(['attacker'])['nb_attack'].sum()
df_attacker = df.set_index(['attacker'])[['latitude_attacker', 'longitude_attacker']]
df_attacker['sum_attacker'] = sum_attacker
df_attacker = df_attacker.reset_index()
df_attacker = df_attacker.drop_duplicates('attacker')
df_attacker['country'] = df_attacker['attacker']
df_attacker = df_attacker.drop(['attacker'], axis = 'columns')

In [60]:
df_attacker = pd.merge(df_attacker, df_code_country, how = 'left', on = ['country'])

In [61]:
df_attacker.head()

,latitude_attacker,longitude_attacker,sum_attacker,country,CODE
0,39.783730,-100.445882,97988,united states,USA
1,39.904211,116.407395,52725,china,CHN
2,49.487197,31.271832,6680,ukraine,UKR
3,52.237989,5.534607,7854,netherlands,NLD
4,22.351115,78.667743,3183,india,IND


In [216]:
colorscale=[[0, "#f7fbff"], [0.25, "#FE96A0"], [0.45, "#FF6F7D"], [0.65, "#FF6F7D"], [0.85, "#DB1702"], [1,"#D90115"]]


data = [ dict(
        type='choropleth',
        colorscale = colorscale,
        autocolorscale = False,
        locations = df_target['CODE'],
        z = np.log(df_target['sum_target']).astype(float),
        text = df_target['country'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 0
            ) ),
        colorbar = dict(
            title = "Cyberattaques <br>échelle logarithmique")
        ) ]

 
layout = dict(
    title = 'Cibles des cyberattaques <br> source : <a href="http://www.norse-corp.com/about-us/who-we-are/">\
Norse </a>',
        
        geo = dict(
            scope='world',
            projection= dict( type='equirectangular'),
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
            showcountries = True
        )
    )
    
fig = dict( data=data, layout=layout )
iplot( fig, filename='d3-cloropleth-map' )

In [217]:
colorscale=[[0, "#f7fbff"], [0.25, "#FE96A0"], [0.45, "#FF6F7D"], [0.65, "#FF6F7D"], [0.85, "#DB1702"], [1,"#D90115"]]

data = [ dict(
        type='choropleth',
        colorscale = colorscale,
        autocolorscale = False,
        locations = df_attacker['CODE'],
        z = np.log(df_attacker['sum_attacker']).astype(float),
        text = df_attacker['country'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 0
            ) ),
        colorbar = dict(
            title = "Cyberattaques <br>échelle logarithmique")
        ) ]

 
layout = dict(
    title = 'Origine des cyberattaques <br> source : <a href="http://www.norse-corp.com/about-us/who-we-are/">\
Norse </a>',
        
        geo = dict(
            scope='world',
            projection= dict( type='equirectangular'),
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
            showcountries = True
        )
    )
    
fig = dict( data=data, layout=layout )
iplot( fig, filename='d3-cloropleth-map' )

In [194]:
zone_h = pd.read_csv('./../../data/DataCountryClean.csv', encoding="latin-1")#.fillna('null')

/Users/raphael/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning:

Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.



In [195]:
zone_h['Country'] = zone_h['Country'].replace(np.nan,'UNKNOWN')

In [196]:
mapping_country = {
    'NL' : 'netherlands', 
    'CN' : 'china', 
    'US' : 'united states', 
    'MX' : 'mexico', 
    'TR' : 'turkey', 
    'PK' : 'pakistan', 
    'HK' : 'hong kong', 
    'VN' : 'vietnam', 
    'RO' : 'roumania', 
    'IT' : 'italia', 
    'KR' : 'south korea',
    'JP' : 'japan', 
    'CH' : 'swiss', 
    'UA' : 'ukraine', 
    'ES' : 'spain', 
    'CZ' : 'czech republic', 
    'BR' : 'brazil', 
    'IN' : 'india', 
    'PH' : 'philippines', 
    'GB' : 'united kingdom', 
    'PL' : 'poland', 
    'FR' : 'france', 
    'IL' : 'israel', 
    'CA' : 'canada', 
    'CO' : 'colombia', 
    'DE' : 'germany', 
    'SA' : 'saudi arabia', 
    'AM' : 'armenia', 
    'BO' : 'bolivia', 
    'BG' : 'bulgaria', 
    'AR' : 'argentina', 
    'TW' : 'taiwan', 
    'MD' : 'moldova',
    'ID' : 'indonesia', 
    'BE' : 'belgium', 
    'TH' : 'thailand', 
    'PT' : 'portugal', 
    'EG' : 'egypt', 
    'CL' : 'chile', 
    'NZ' : 'new zealand', 
    'RU' : 'russia', 
    'CY' : 'cyprus', 
    'BM' : 'bermuda', 
    'MN' : 'mongolia',
    'IR' : 'ireland', 
    'BD' : 'bangladesh', 
    'SI' : 'slovenia', 
    'VE' : 'venezuela', 
    'AU' : 'australia', 
    'AE' : 'united arab emirates',
    'NO' : 'norway',
    'SG' : 'singapore',
    'IE' : 'ireland',
    'IS' : 'iceland',
    'AT' : 'austria'    
}

In [197]:
mapping_country = list(mapping_country.values())

In [198]:
zone_h['Country'] = zone_h['Country'].apply(lambda x : x.lower())

In [199]:
def referenced_country(x):
    if x in mapping_country : 
        return x
    else : 
        return np.nan

In [200]:
zone_h['Country'] = zone_h['Country'].apply(lambda x : referenced_country(x))

In [201]:
zone_h[zone_h['retrieveCountry'].notnull()].head()

,Date,Notify,URL,IP,Country,OS,WebServer,Encoding,Lang,company,retrieveCountry,retrieveIndustry
9,1998-01-07,D.A.M.M.,http://www.unicef.org,NaN,NaN,Unknown,Unknown,windows-1252,NaN,unicef,united states,NaN
104,1999-06-15,Forpaxe,http://insite.net,NaN,NaN,Unknown,Unknown,utf-8,NaN,insite,canada,health
119,1999-06-21,V3|uW3,http://www.lancia.nl,NaN,NaN,Unknown,NaN,NaN,NaN,lancia,italy,automotive
121,1999-06-21,BiS,http://data3.gmu.edu,NaN,NaN,Unknown,Unknown,NaN,NaN,data3,australia,information
295,1999-08-03,bl0w,http://www.symantec.com,NaN,NaN,Unknown,Unknown,iso-8859-1,NaN,symantec,united states,computer


In [202]:
zone_h['retrieveCountry'] = zone_h['retrieveCountry'].replace('unknown ', np.nan)
zone_h['retrieveCountry'] = zone_h['retrieveCountry'].fillna(zone_h['Country'])

In [203]:
zone_h['retrieveCountry'] = zone_h['retrieveCountry'].replace('unknown', np.nan)

In [207]:
len(zone_h) - zone_h['retrieveCountry'].isnull().sum()

117750